In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
#import matplotlib.pyplot as plt
import time
import os
import copy
import segmentation_models_pytorch as smp

In [1]:
import os 

os.listdir('data/files/dataset/256/train/input/Goal_map')[:5]

['112_25_Denver_0_256.map',
 '150_162_London_0_256.map',
 '121_40_NewYork_0_256.map',
 '228_29_Berlin_1_256.map',
 '243_117_Boston_0_256.map']

In [2]:
os.listdir('data/files/dataset/256/train/output')[:5]

['112_25_Denver_0_256.map',
 '150_162_London_0_256.map',
 '121_40_NewYork_0_256.map',
 '228_29_Berlin_1_256.map',
 '243_117_Boston_0_256.map']

In [3]:
for line in os.listdir('data/files/dataset/256/train/input/Goal_map')[:5]:
    line = line.split('_')[2:]
    file_name = line[0]
    for el in line[1:]:
        file_name += '_' + el
    print('data/files/dataset/256/train/input/input_map/' + file_name)

data/files/dataset/256/train/input/input_map/Denver_0_256.map
data/files/dataset/256/train/input/input_map/London_0_256.map
data/files/dataset/256/train/input/input_map/NewYork_0_256.map
data/files/dataset/256/train/input/input_map/Berlin_1_256.map
data/files/dataset/256/train/input/input_map/Boston_0_256.map


In [ ]:
import numpy as np
from torch.utils.data import Dataset
import os

def read_data(path):
    data = []
    with open(path, 'r') as fopen:
        for line in fopen:
            data.append([float(x) for x in line.strip('\n').split()])
            
    return data



class MyDataset(Dataset):
    def __init__(self, path):
        self.input_path1 = path + 'input/input_map/'
        self.input_path2 = path + 'input/Goal_map/'
        self.target_path = path + 'output/'
        
    def __len__(self):
        return len(os.listdir(self.target_path))
    
    def __getitem__(self, idx):
        
        in_path2 = self.input_path2 + os.listdir(self.input_path2)[idx]
        
        line = os.listdir(self.input_path2)[idx].split('_')[2:]
        file_name = line[0]
        for el in line[1:]:
            file_name += '_' + el
        in_path1 = self.input_path1 + file_name
        
        tar_path = self.target_path + os.listdir(self.target_path)[idx]
        
        in_data1 = read_data(in_path1)
        in_data2 = read_data(in_path2)
        in_data = np.array([in_data1, in_data2], dtype='float32')
        
        tar_data = read_data(tar_path)
        tar_data = np.array([tar_data], dtype='float32')
        
        return in_data, tar_data

In [ ]:
path = 'data/files/dataset/256/train/'
input_path1 = path + 'input/input_map/'
input_path2 = path + 'input/Goal_map/'
target_path = path + 'output/'


def delete():
    
    for i, file_name in enumerate(os.listdir(input_path2)):

        print('iter = {}'.format(i), end='\r')
        in_path2 = input_path2 + file_name

        tar_path = target_path + file_name

        in_data2 = read_data(in_path2)
        tar_data = read_data(tar_path)
        if len(in_data2) < 256 or len(tar_data) < 256:
            print(in_path2, len(in_data2))
            os.system('rm ' + in_path2)
            os.system('rm ' + tar_path)

    print('DONE!!!!!!!!!!')
        
#delete()

In [ ]:
def check(path1, path2):
    for x1, x2 in zip(os.listdir(path1), os.listdir(path2)):
        if x1 != x2:
            print('NO!!!!')
            break
    print('YES!')

path1 = 'data/files/train/input/Goal_map'
path2 = 'data/files/train/output'
            
#check(path1, path2)

## Set no grad encoder

In [ ]:
def set_no_grad_encoder(model):
    for parameter in model.encoder.parameters():
        parameter.requires_grad_(False)

## Prepare data

In [ ]:

def Get_Dataset(train_path, test_path, batch_size):
    train_dataset = MyDataset(train_path)
    test_dataset = MyDataset(test_path)

    dataloaders = {'train' : DataLoader(train_dataset, batch_size=batch_size, shuffle=True),
                  'val' : DataLoader(test_dataset, batch_size=batch_size, shuffle=True)}

    dataset_sizes = {'train' : len(train_dataset),
                    'val' : len(test_dataset)}

    
    return dataloaders, dataset_sizes

# Train fuction

In [1]:
def train_model(model, criterion, optimizer, scheduler, allowed_indexes=False, num_epochs=100):
    since = time.time()

    global best_model_wts
    global model_save_path
    #best_acc = 0.0
    best_loss = np.inf

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            #running_corrects = 0

            
            # Iterate over data.
            it = 0
            for inputs, labels in dataloaders[phase]:
                
                print('iter = {}'.format(it), end='\r')
                it += inputs.size(0)
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    
                    outputs = model(inputs)
                    
                    if allowed_indexes:
                        indexes = inputs[:, :1] == 0
                        outputs = outputs[indexes]
                        labels = labels[indexes]
                    #_, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                #running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            #epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f}'.format(
                phase, epoch_loss))

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model, model_save_path)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


### Set model, criterion, optimizer, lr_scheduler

In [ ]:
def Get_MCOS():

    model = smp.Unet('resnet34', encoder_weights='imagenet', in_channels=2)
    set_no_grad_encoder(model)

    model = model.to(device)

    criterion = nn.MSELoss()

    # Observe that all parameters are being optimized
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    return model, criterion, optimizer, exp_lr_scheduler

## Set device and batch_size

In [ ]:
batch_size = 32
epoch = 50

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Trainning using all output

### map_size 256

In [ ]:
# set paths
train_path = 'data/files/dataset/256/train/'
test_path = 'data/files/dataset/256/test/'
model_save_path = 'model/model_256.pt'

# set dataloaders
dataloaders, dataset_sizes = Get_Dataset(train_path, test_path, batch_size)
model, criterion, optimizer, exp_lr_scheduler = Get_MCOS()


best_model_wts = copy.deepcopy(model.state_dict())

model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=epoch)



### map_size 128

In [ ]:
# set paths
train_path = 'data/files/dataset/128/train/'
test_path = 'data/files/dataset/128/test/'
model_save_path = 'model/model_128.pt'

# set dataloaders
dataloaders, dataset_sizes = Get_Dataset(train_path, test_path, batch_size)
model, criterion, optimizer, exp_lr_scheduler = Get_MCOS()


best_model_wts = copy.deepcopy(model.state_dict())

model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=epoch)



### map_size 64

In [ ]:
# set paths
train_path = 'data/files/dataset/64/train/'
test_path = 'data/files/dataset/64/test/'
model_save_path = 'model/model_64.pt'

# set dataloaders
dataloaders, dataset_sizes = Get_Dataset(train_path, test_path, batch_size)
model, criterion, optimizer, exp_lr_scheduler = Get_MCOS()


best_model_wts = copy.deepcopy(model.state_dict())

model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=epoch)



## Trainning using outputs with index in which input values is zero

### map_size 256

In [ ]:
# set paths
train_path = 'data/files/dataset/256/train/'
test_path = 'data/files/dataset/256/test/'
model_save_path = 'model/model_256_zero.pt'

# set dataloaders
dataloaders, dataset_sizes = Get_Dataset(train_path, test_path, batch_size)
model, criterion, optimizer, exp_lr_scheduler = Get_MCOS()


best_model_wts = copy.deepcopy(model.state_dict())

model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, allowed_indexes=True,
                       num_epochs=epoch)


### map_size 128

In [ ]:
# set paths
train_path = 'data/files/dataset/128/train/'
test_path = 'data/files/dataset/128/test/'
model_save_path = 'model/model_128.pt'

# set dataloaders
dataloaders, dataset_sizes = Get_Dataset(train_path, test_path, batch_size)
model, criterion, optimizer, exp_lr_scheduler = Get_MCOS()


best_model_wts = copy.deepcopy(model.state_dict())

model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, allowed_indexes=True,
                       num_epochs=epoch)

### map_size 64

In [ ]:
# set paths
train_path = 'data/files/dataset/64/train/'
test_path = 'data/files/dataset/64/test/'
model_save_path = 'model/model_64.pt'

# set dataloaders
dataloaders, dataset_sizes = Get_Dataset(train_path, test_path, batch_size)
model, criterion, optimizer, exp_lr_scheduler = Get_MCOS()


best_model_wts = copy.deepcopy(model.state_dict())

model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, allowed_indexes=True,
                       num_epochs=epoch)